In [1]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F

from fastai import *
from fastai.basic_train import *
from fastai.basic_data import *
from fastai.data_block import *
from fastai.torch_core import *
from fastai.train import *
from fastai.callback import *
from fastai.distributed import *

In [2]:
def preprocess(fname, type_index=None):
    t  = pd.read_csv(fname)
    s  = pd.read_csv('structures.csv')
    
    has_y = 'scalar_coupling_constant' in t.columns

    if has_y:
        # atom-atom level
        # molecule_name,atom_index_0,atom_index_1,type,fc,sd,pso,dso
        scalar_couplings = pd.read_csv('scalar_coupling_contributions.csv') # fc,sd,pso,dso

        # atom level
        # molecule_name,atom_index,XX,YX,ZX,XY,YY,ZY,XZ,YZ,ZZ
        magnetic_shielding = pd.read_csv('magnetic_shielding_tensors.csv')
        # molecule_name,atom_index,mulliken_charge
        mulliken_charges = pd.read_csv('mulliken_charges.csv')

        # molecule level
        # molecule_name,X,Y,Z
        dipole_moments = pd.read_csv('dipole_moments.csv')
        # molecule_name,potential_energy
        potential_energy = pd.read_csv('potential_energy.csv')

    t['molecule_index'] = pd.factorize(t['molecule_name'])[0] + t['id'].min()
    # make sure we use the same indexes in train/test (test needs to provide type_index)
    if type_index is not None:
        t['type_index'] = pd.factorize(pd.concat([pd.Series(type_index),t['type']]))[0][len(type_index):]
    else:
        t['type_index'] = pd.factorize(t['type'])[0]
    s = pd.concat([s,pd.get_dummies(s['atom'])], axis=1)

    max_items = 785836 if has_y else 422550
    max_atoms = int(s.atom_index.max() + 1)

    if has_y:
        contributions = ['fc','sd','pso','dso']
        magnetic_tensors = ['XX','YX','ZX','XY','YY','ZY','XZ','YZ','ZZ']
        XYZ = ['X','Y','Z']
    xyz = ['x', 'y', 'z']
    a_hot = ['C','F','H','N','O']
    
    x_xyz   = np.zeros((max_items,len(xyz),  max_atoms), dtype=np.float32)
    x_a_hot = np.zeros((max_items,len(a_hot),max_atoms), dtype=np.float32)
    x_type  = np.zeros((max_items,1,         max_atoms), dtype=np.float32)

    if has_y:
        y_scalar   = np.zeros((max_items,len(contributions)   ,max_atoms), dtype=np.float32)
        y_magnetic = np.zeros((max_items,len(magnetic_tensors),max_atoms), dtype=np.float32)
        y_mulliken = np.zeros((max_items,1                    ,max_atoms), dtype=np.float32)

        y_dipole   = np.zeros((max_items,len(XYZ)), dtype=np.float32)
        y_potential= np.zeros((max_items,1              ), dtype=np.float32)

        y_magnetic[...] = np.nan
        y_mulliken[...] = np.nan
    else:
        xt_ids = np.zeros((max_items, max_atoms), dtype=np.int32)


    m = np.zeros((max_items,), dtype=np.int32)
    i = j = 0
    
    for (m_name, m_index) ,m_group in tqdm(t.groupby(['molecule_name', 'molecule_index'])):
        ss = s[s.molecule_name==m_name]
        n_atoms = len(ss)
        if has_y:
            magnetic = magnetic_shielding[
                    (magnetic_shielding['molecule_name']==m_name)][magnetic_tensors].values.T

            mulliken = mulliken_charges[
                    (mulliken_charges['molecule_name']==m_name)]['mulliken_charge'].values.T

            scs = scalar_couplings[scalar_couplings['molecule_name']==m_name]
            
            y_dipole[j,:]= dipole_moments[dipole_moments['molecule_name']==m_name][XYZ].values
            y_potential[j,:]=potential_energy[
                potential_energy['molecule_name']==m_name]['potential_energy'].values
        
        for a_name,a_group in m_group.groupby('atom_index_0'):
            
            ref_a = ss[ss['atom_index']==a_name]
            
            x_xyz[i] = 0.
            x_a_hot[i] = ref_a[a_hot].values.T
            x_type[i] = -1

            x_xyz[i,:,:n_atoms] = (ss[xyz].values-ref_a[xyz].values).T  # xyz 
            x_a_hot[i,:,:n_atoms] = ss[a_hot].T                  # a_hot
            x_type[i,0,a_group['atom_index_1']] = a_group['type_index']  # type 
            
            if has_y:
                y_scalar[i,:,a_group['atom_index_1']] = scs[scs['atom_index_0']==a_name][contributions]
                y_magnetic[i,:,:n_atoms] = magnetic
                y_mulliken[i,:,:n_atoms] = mulliken
            else:
                xt_ids[i,a_group['atom_index_1']] = a_group['id']  

            m[i] = m_index
            i+=1
        j += 1
    assert i == max_items
    print(i,max_items)
    if has_y:
        return x_xyz,x_a_hot,x_type, m , y_scalar, y_magnetic, y_mulliken, y_dipole, y_potential
    else:
        return x_xyz,x_a_hot,x_type, m, xt_ids

In [3]:
train_fname = Path('train.npz')
try:
    npzfile = np.load(train_fname)
    x_xyz = npzfile['x_xyz']
    x_a_hot = npzfile['x_a_hot']
    x_type = npzfile['x_type']
    y_scalar = npzfile['y_scalar']
    y_magnetic = npzfile['y_magnetic']
    y_mulliken = npzfile['y_mulliken']
    y_dipole = npzfile['y_dipole']
    y_potential = npzfile['y_potential']
    m = npzfile['m']
    max_items, max_atoms = x_xyz.shape[0], x_xyz.shape[-1]
except:
    x_xyz,x_a_hot,x_type, m , y_scalar, y_magnetic, y_mulliken, y_dipole, y_potential = \
        preprocess(train_fname.with_suffix('.csv'))
    np.savez(train_fname, 
             x_xyz=x_xyz,
             x_a_hot=x_a_hot,
             x_type=x_type,
             y_scalar=y_scalar,
             y_magnetic=y_magnetic,
             y_mulliken=y_mulliken,
             y_dipole=y_dipole,
             y_potential=y_potential,
             m=m)
n_types = int(x_type[~np.isnan(x_type)].max() + 1)

In [4]:
use_memmap = True
try:
    load_fn = np.load if not use_memmap else partial(np.lib.format.open_memmap, mode='r')
    x_coulombmat = load_fn('x_coulombmat32.npy')
except:
    x_coulombmat = np.load('x_coulombmat.npy', allow_pickle=True)
    x_coulombmat = np.array(x_coulombmat.tolist()).astype(np.float32)
    np.save('x_coulombmat32.npy', x_coulombmat)

In [5]:
test_fname = Path('test.npz')
try:
    npzfile = np.load(test_fname)
    xt_xyz = npzfile['x_xyz']
    xt_a_hot = npzfile['x_a_hot']
    xt_type = npzfile['x_type']
    mt = npzfile['m']
    #xt_ids = npzfile['x_ids']
except:
    train_csv = pd.read_csv(train_fname.with_suffix('.csv'))
    xt_xyz,xt_a_hot,xt_type,mt,xt_ids = \
        preprocess(test_fname.with_suffix('.csv'), type_index = pd.factorize(train_csv['type'])[1])
    np.savez(test_fname, 
             x_xyz=xt_xyz,
             x_a_hot=xt_a_hot,
             x_type=xt_type,
             m=mt,
             x_ids=xt_ids)
    
try:
    xt_ids = np.load('xt_ids.npy')
except:
    print("ASK PAVEL")
    assert False

In [6]:
[v.shape for v in [x_xyz,x_a_hot,x_type, y_scalar, y_magnetic, y_mulliken, y_dipole, y_potential, m]]

[(785836, 3, 29),
 (785836, 5, 29),
 (785836, 1, 29),
 (785836, 4, 29),
 (785836, 9, 29),
 (785836, 1, 29),
 (785836, 3),
 (785836, 1),
 (785836,)]

In [7]:
# TODO: Do norm in FASTAI instead of here
# TOCHECK: Filter only valid atoms (otherwise repeated atoms may skew stats)
#xyz_mean, xyz_std = x_xyz.mean(axis=(0,2), keepdims=True),  x_xyz.std(axis=(0,2), keepdims=True)
#x_xyz  = (x_xyz  - xyz_mean) / xyz_std
#xt_xyz = (xt_xyz - xyz_mean) / xyz_std

In [8]:
[v.shape for v in [xt_xyz,xt_a_hot,xt_type, mt]]

[(422550, 3, 29), (422550, 5, 29), (422550, 1, 29), (422550,)]

In [9]:
class MoleculeItem(ItemBase):
    def __init__(self,i,xyz,a_hot,type,coulomb): 
        self.i,self.xyz,self.a_ahot,self.type,self.coulomb = i,xyz,a_hot,type,coulomb
        self.data = [torch.cat([Tensor(xyz), Tensor(self.a_ahot),Tensor(self.type)], dim=0), 
                     Tensor(coulomb)]
    def __str__(self):
        # TODO: count n_atoms correctly. 
        n_atoms = np.count_nonzero(np.sum(np.absolute(self.xyz), axis=0))+1
        n_couplings = np.sum((self.type!=-1))
        return f'{self.i} {n_atoms} atoms {n_couplings} couplings'
    
    def apply_tfms(self, tfms:Collection, **kwargs):
        x = self.clone()
        #print(tfms)
        for t in tfms:
            if t: x.data = t(x.data)
        return x
    
    def clone(self):
        return self.__class__(self.i,self.xyz,self.a_ahot,self.type,self.coulomb)
    
class ScalarCouplingItem(ItemBase):
    def __init__(self,scalar,magnetic,mulliken,dipole,potential,**kwargs): 
        self.scalar,self.magnetic,self.mulliken,self.dipole,self.potential = \
            scalar,magnetic,mulliken,dipole,potential
        #self.data = Tensor(np.sum(scalar, axis=0))
        self.data = Tensor(scalar)
    def __str__(self):
        res, spacer, n_couplings = '', '', 0
        for s in self.data.sum(dim=0):
            if s==0.: spacer = ' * '
            else: 
                res += f'{spacer}{s:.4f}'
                spacer = ' '
                n_couplings +=1
        return f'{n_couplings}: {res}'
    def __hash__(self): return hash(str(self))

In [10]:
def LMAEMaskedLoss(input_outputs,target,input_transform_w=0,feature_transform_w=0.,contrib_w=1e-1):
    input, output, trans, trans_feat = input_outputs
    loss = 0.
    n = 0
    for type in range(n_types):
        mask = (input[:,8,:] == type)
        if mask.sum() > 0:
            _output,_target = output.transpose(1,2)[mask], target.transpose(1,2)[mask] # scalars at the end
            # LMAE scalar
            loss += torch.log((_output.sum(dim=-1) - _target.sum(dim=-1)).abs().mean()+1e-9)
            # LMAE scalar contributions
            for i_contrib in range(_output.shape[-1]):
                                loss += contrib_w * torch.log((_output[...,i_contrib] - _target[...,i_contrib]).abs().mean()+1e-9)
            n+=1
    loss /= n
    if input_transform_w   > 0: loss += feature_transform_regularizer(trans)*input_transform_w
    if feature_transform_w > 0: loss += feature_transform_regularizer(trans_feat)*feature_transform_w
    return loss

In [11]:
class ScalarCouplingList(ItemList):
    def __init__(self, items:Iterator, **kwargs):
        super().__init__(items, **kwargs)
        self.c = 4
        self.loss_func = LMAEMaskedLoss

    def get(self, i):
        o = super().get(i)
        return ScalarCouplingItem(*o)

    def reconstruct(self,t): return 0; # TODO for viz !!!! ScalarCouplingItem(t.numpy())

In [12]:
#https://github.com/facebookresearch/QuaterNet/blob/master/common/quaternion.py
def qrot(q, v):
    """
    Rotate vector(s) v about the rotation described by quaternion(s) q.
    Expects a tensor of shape (*, 4) for q and a tensor of shape (*, 3) for v,
    where * denotes any number of dimensions.
    Returns a tensor of shape (*, 3).
    """
    assert q.shape[-1] == 4
    assert v.shape[-1] == 3
    assert q.shape[:-1] == v.shape[:-1]
    
    
    original_shape = list(v.shape)
    q = q.view(-1, 4)
    v = v.view(-1, 3)
    
    qvec = q[:, 1:]
    uv = torch.cross(qvec, v, dim=1)
    uuv = torch.cross(qvec, uv, dim=1)
    return (v + 2 * (q[:, :1] * uv + uuv)).view(original_shape)

def random_rotation(data):
    x_xyz = data[0][:3].transpose(0,1)
    r = torch.rand(3)
    sq1_v1,sqv1,v2_2pi,v3_2pi = torch.sqrt(1-r[:1]),torch.sqrt(r[:1]),2*math.pi*r[1:2],2*math.pi*r[2:3]
    q = torch.cat([sq1_v1*torch.sin(v2_2pi), sq1_v1*torch.cos(v2_2pi), 
                   sqv1  *torch.sin(v3_2pi), sqv1  *torch.cos(v3_2pi)], dim=0).unsqueeze(0)
    x_xyz = qrot(q.expand(x_xyz.shape[0],-1), x_xyz).squeeze(0).transpose(0,1)
    return (torch.cat((x_xyz, data[0][3:]), dim=0), data[1])

def smooth_atom_one_hot(data):
    r = torch.rand(5,5)
    r[:,0] = r[:,0]*.2 + 0.8
    s = 1 - r[:,0]
    rs = r[:,1:].sum(dim=1,keepdim=True)
    r[:,1:] *= s.unsqueeze(-1)/rs
    for i in range(5):
        r[i] = r[i].roll(i,dims=0)
    a_hot = data[0][3:-1].argmax(dim=0)
    data[0][3:-1] = r[a_hot].transpose(0,1)
    return data

In [13]:
data = ItemList(items=(MoleculeItem(i,*v) for i,v in enumerate(zip(x_xyz,x_a_hot,x_type, x_coulombmat))),
                label_cls=ScalarCouplingItem)

In [14]:
from sklearn.model_selection import train_test_split
_, idx_valid_split = train_test_split(range(m.max()+1), test_size=0.1, random_state=13)
idx_valid_split = np.argwhere(np.isin(m, idx_valid_split)).flatten()

In [15]:
data = data.split_by_idx(idx_valid_split)

In [16]:
data = data.label_from_func(
    func=lambda o: (y_scalar[o.i], y_magnetic[o.i], y_mulliken[o.i], y_dipole[o.i], y_potential[o.i]),
    label_cls=ScalarCouplingList)

In [17]:
try:
    xt_coulombmat = load_fn('xt_coulombmat32.npy')
except:
    xt_coulombmat = np.load('xt_coulombmat.npy', allow_pickle=True)
    xt_coulombmat = np.array(xt_coulombmat.tolist()).astype(np.float32)
    np.save('xt_coulombmat32.npy', xt_coulombmat)

In [18]:
data.add_test(
    ItemList(items=(MoleculeItem(i,*v) for i,v in enumerate(zip(xt_xyz,xt_a_hot,xt_type,xt_coulombmat)))))

LabelLists;

Train: LabelList (707619 items)
x: ItemList
0 5 atoms 4 couplings,1 5 atoms 3 couplings,2 5 atoms 2 couplings,3 5 atoms 1 couplings,4 4 atoms 3 couplings
y: ScalarCouplingList
4: 84.8076 * -11.2569 -11.2549 -11.2543,3: 84.8074 * -11.2542 -11.2548,2: 84.8093 * -11.2543,1: 84.8095,3: 32.6888 * -11.1867 -11.1757
Path: .;

Valid: LabelList (78217 items)
x: ItemList
9 8 atoms 7 couplings,10 8 atoms 6 couplings,11 8 atoms 5 couplings,12 8 atoms 4 couplings,13 8 atoms 3 couplings
y: ScalarCouplingList
7: 83.5430 -2.3783 * -11.7004 -11.6979 3.2528 13.6913 3.2521,6: 83.5417 -2.3786 * -11.6996 13.6924 3.2525 3.2527,5: 83.5484 -2.3772 * 3.2524 3.2524 13.6921,4: -2.3788 83.5418 * -11.7004 -11.6993,3: -2.3785 83.5430 * -11.6976
Path: .;

Test: LabelList (422550 items)
x: ItemList
0 4 atoms 3 couplings,1 4 atoms 2 couplings,2 9 atoms 4 couplings,3 9 atoms 3 couplings,4 9 atoms 2 couplings
y: EmptyLabelList
,,,,
Path: .

In [19]:
data = data.transform(([random_rotation, smooth_atom_one_hot], [random_rotation, smooth_atom_one_hot]))

# IMPORTANT IF/WHEN DOING TTA ch

#data = data.transform(([random_rotation, smooth_atom_one_hot], None))

In [20]:
data=data.databunch()

In [21]:
# run this a few times to see rotations at work (atom xyz should change)
#data.test_ds[0][0].data[0][:3]

In [22]:
class ChemCallback(Callback):
    def __init__(self): super().__init__()
    def on_batch_begin(self,**kwargs):
        "Save the last_input (i.e. current input) for on_loss_begin_callback"
        self.last_input = kwargs['last_input']
    def on_loss_begin(self, last_output:Tuple[Tensor,Tensor,Tensor], **kwargs):
        "Add last_input to last_output, i.e. input to loss function b/c we need it"
        return {'last_output': (self.last_input[0], *last_output)}

In [23]:
# TODO REFACTOR
class Quaternion(Module):
    def __init__(self, in_channels = 3):
        self.conv1 = torch.nn.Conv1d(in_channels, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 3)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = torch.sigmoid(self.fc3(x))
        
        #http://planning.cs.uiuc.edu/node198.html
        sq1_v1,sqv1,v2_2pi,v3_2pi = torch.sqrt(1-x[:,:1]),torch.sqrt(x[:,:1]),2*math.pi*x[:,1:2],2*math.pi*x[:,2:3]
        q = torch.cat([sq1_v1*torch.sin(v2_2pi), sq1_v1*torch.cos(v2_2pi), 
                       sqv1  *torch.sin(v3_2pi), sqv1  *torch.cos(v3_2pi)], dim=1)
        return q
    
# https://github.com/eladhoffer/fix_your_classifier/blob/master/fixed_proj.py
class LinearProjection(Module):

    def __init__(self, input_size, output_size, bias=True, init_scale=10):
        if init_scale is not None:
            self.weight = nn.Parameter(torch.Tensor(1).fill_(init_scale))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(output_size).fill_(0))
        self.proj = Variable(torch.Tensor(
            output_size, input_size), requires_grad=False)
        torch.manual_seed(123)
        nn.init.orthogonal_(self.proj)

    def forward(self, x):
        w = self.proj.type_as(x)
        x = x / x.norm(2, -1, keepdim=True)
        out = nn.functional.linear(x, w)
        if hasattr(self, 'weight'): out = out * self.weight
        if hasattr(self, 'bias'):   out = out + self.bias.view(1, -1)
        return out

# from https://github.com/fxia22/pointnet.pytorch/blob/master/pointnet/model.py
class STNkd(Module):
    def __init__(self, pools, channels, add_eye, last_fc=nn.Linear):
        k, c_filters, fc_filters = channels[0][0], channels[0][1:], channels[1]
        self.conv1 = torch.nn.Conv1d(k, c_filters[0], 1)
        self.conv2 = torch.nn.Conv1d(c_filters[0], c_filters[1], 1)
        self.conv3 = torch.nn.Conv1d(c_filters[1], c_filters[2], 1)
        self.fc1 = nn.Linear(c_filters[2]*len(pools), fc_filters[0])
        self.fc2 = nn.Linear(fc_filters[0], fc_filters[1])
        self.fc3 = last_fc(fc_filters[1], k*k)
        # TODO REFACTOR
        self.relu = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(c_filters[0])
        self.bn2 = nn.BatchNorm1d(c_filters[1])
        self.bn3 = nn.BatchNorm1d(c_filters[2])
        self.bn4 = nn.BatchNorm1d(fc_filters[0])
        self.bn5 = nn.BatchNorm1d(fc_filters[1])
        self.pools = [pool(1) for pool in pools]

        self.k = k
        self.add_eye = add_eye

    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.cat([pool(x).squeeze(-1) for pool in self.pools], dim=-1)    
        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x).view(-1, self.k, self.k)
        if self.add_eye: x += torch.eye(self.k, dtype=x.dtype, device=x.device).unsqueeze(0)
        return x

class PointNetfeat(Module):
    def __init__(self, pools, C, quaternion_transform,
                 input_transform, feature_transform, add_eye=True):
        self.qrot = Quaternion() if quaternion_transform else None
        self.stn = STNkd(pools, channels = input_transform, add_eye=add_eye) if input_transform else None        
        self.convs = nn.ModuleList([nn.Sequential(
            nn.Conv1d(C[i], C[i+1], 1), nn.BatchNorm1d(C[i+1]), nn.ReLU(), ) for i in range(len(C)-1)])
        self.fstn = STNkd(pools, channels = feature_transform, add_eye=add_eye, 
                         last_fc = nn.Linear) if feature_transform else None
        self.pools = [pool(1) for pool in pools]

    def forward(self, x):
        batchsize, _, n_pts = x.shape
        if self.qrot is not None:
            x_xyz = x[:,:3,...]
            q = self.qrot(x_xyz).unsqueeze(1).expand(-1,n_pts,-1).contiguous()
            x_xyz = x_xyz.transpose(1,2).contiguous()
            x[:,:3,:] = qrot(q,x_xyz).transpose(2,1)
        if self.stn is not None: # TODO add xyz_
            x_xyz = x[:,:3,...].clone()
            trans = self.stn(x_xyz)
            x[:,:3,:] = torch.bmm(trans, x_xyz)
        else:
            trans = torch.eye(1, dtype=x.dtype, device=x.device).view(1,1).expand((batchsize,-1))
        for conv in self.convs[:2]:
            x = conv(x)

        if self.fstn is not None:
            trans_feat = self.fstn(x)
            x = torch.bmm(trans_feat, x)
        else:
            trans_feat = torch.eye(1, dtype=x.dtype, device=x.device).view(1,1).expand((batchsize,-1))

        pointfeat = x
        for conv in self.convs[2:]:
            x = conv(x)
        x = torch.cat([pool(x).squeeze(-1) for pool in self.pools], dim=-1)    
        return pointfeat, x, trans, trans_feat
    
class PointNetDenseReg(Module):
    def __init__(self, feat_channels, seg_channels, 
                 quaternion_transform, input_transform, feature_transform, add_eye, 
                 pools = [nn.AdaptiveMaxPool1d], dropout_rate = 0.):
        self.feat = PointNetfeat(pools,
                                 C = feat_channels,
                                 quaternion_transform = quaternion_transform,
                                 input_transform = input_transform, 
                                 feature_transform=feature_transform,
                                 add_eye=add_eye)
        #self.feat_c = PointNetfeat(pools,
        #                         C = feat_channels,
        #                         quaternion_transform = quaternion_transform,
        #                         input_transform = input_transform, 
        #                         feature_transform=feature_transform,
        #                         add_eye=add_eye)
        C = seg_channels
        
        C.insert(0, 2*feat_channels[-1]*len(pools) + feat_channels[1])        
        
        self.seg_convs = nn.ModuleList([nn.Sequential(
            nn.Conv1d(C[i], C[i+1], 1), nn.BatchNorm1d(C[i+1]), nn.ReLU(),) for i in range(len(C)-2)])
        self.seg_convs.append(nn.Conv1d(C[-2], C[-1], 1))
            
        coulomb_C = [feat_channels[2],128,1024]
        
        self.coulomb_convs = nn.ModuleList([nn.Sequential(
            nn.Conv1d(coulomb_C[i], coulomb_C[i+1], 1), nn.BatchNorm1d(coulomb_C[i+1]), nn.ReLU(),) 
                                          for i in range(len(coulomb_C)-1)])

        self.dropout = nn.Dropout(p=dropout_rate) if dropout_rate > 0. else None
        self.pools = [pool(1) for pool in pools]
        
    def forward(self, x, coulomb):
        batchsize, _, n_pts = x.shape
        pointfeat, globalfeat, trans, trans_feat = self.feat(x)
        x_c = torch.matmul(pointfeat, coulomb)
        for coulomb_conv in self.coulomb_convs[:3]:
            x_c = coulomb_conv(x_c)
        x_c = torch.cat([pool(x_c) for pool in self.pools], dim=-1)   
        for coulomb_conv in self.coulomb_convs[3:]:
            x_c = coulomb_conv(x_c)

        #print(globalfeat.shape, pointfeat.shape, x_c.shape)
        x = torch.cat([globalfeat.unsqueeze(-1).expand(-1, -1, n_pts), 
                       pointfeat,
                      #], 1)
                       x_c.expand(-1, -1, n_pts)], 1)
        #print(x.shape)
        for seg_conv in self.seg_convs[:-1]:
            x = seg_conv(x)
        if self.dropout is not None: x = self.dropout(x)
        x = self.seg_convs[-1](x)
        return x, trans, trans_feat

def feature_transform_regularizer(trans): 
    batchsize,d  = trans.shape[0], trans.shape[1]
    I = torch.eye(d, dtype=trans.dtype, device=trans.device).unsqueeze(0)
    loss = torch.mean(torch.norm(torch.bmm(trans, trans.transpose(2,1)) - I, dim=(1,2)))
    return loss

In [24]:
net, learner = None,None
gc.collect()
torch.cuda.empty_cache()
net = PointNetDenseReg(feat_channels = [9,64,64,128,1024],
                       seg_channels  = [512,256,128,4],
                       quaternion_transform=False, 
                       input_transform=[[3, 64,128,1024], [512,256]], 
                       feature_transform=[[64, 64,128,1024], [512,256]],
                       add_eye=True)
learner = Learner(data,net, callbacks=[ChemCallback()],
                  loss_func=partial(LMAEMaskedLoss,input_transform_w=0., feature_transform_w=1e-3),
                  metrics=[ partial(LMAEMaskedLoss,input_transform_w=0., feature_transform_w=0.,contrib_w=0.)])
data.batch_size = 4096*4

In [25]:
sub_fname = "loss-4.1809val-1.9292" # '600k' #'val-1.7450' # 'loss-1.6714val-1.5721'

In [26]:
try:
    print(f"Attempting to load: {sub_fname}... ", end="")
    learner.load(sub_fname, strict=False,with_opt=False)
    print("Loaded")
except:
    print("NOT loaded!")
    pass
    

Attempting to load: loss-4.1809val-1.9292... 

/home/antor/miniconda3/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type PointNetDenseReg. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/antor/miniconda3/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type PointNetfeat. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/antor/miniconda3/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type STNkd. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Loaded


In [ ]:
learner.summary()

In [ ]:
learner.model

In [27]:
learner = learner.to_parallel()#.to_fp16()

In [ ]:
data.batch_size = int(4096*1.8)
learner.lr_find()
learner.recorder.plot(suggestion=True)

In [ ]:
learner.fit_one_cycle(1000, 1e-2, moms=(0.75,0.70))

epoch,train_loss,valid_loss,LMAEMaskedLoss,time
0,-3.960701,-3.424191,-1.852465,01:02
1,-3.962877,-3.434547,-1.882199,00:58
2,-3.969479,-3.474532,-1.895448,00:58
3,-3.967575,-3.437969,-1.862027,00:59
4,-3.960371,-3.449721,-1.885986,00:58
5,-3.961676,-3.451243,-1.879852,00:58
6,-3.965587,-3.454905,-1.879316,00:58
7,-3.975518,-3.429182,-1.870930,00:58
8,-3.968286,-3.451897,-1.886748,00:58
9,-3.968139,-3.469301,-1.887624,00:58


In [ ]:
learner.recorder.plot_lr(show_moms=True)

In [ ]:
learner.recorder.plot_losses()

In [ ]:
learner.fit(1000, 2e-3)#, moms=(0.75,0.70))

In [28]:
data.batch_size = 4096
learner.to_fp32()
val = learner.validate(metrics=[partial(LMAEMaskedLoss,input_transform_w=0., feature_transform_w=0.,contrib_w=0.)])[1]
print(val)

tensor(-1.9321)


In [ ]:
sub_fname = f'loss{learner.recorder.losses[-1]:.04f}val{val:.04f}'
#sub_fname = f'val{val:.04f}'
learner.save(sub_fname)
print(sub_fname)

In [29]:
TTA_N = 100

sub = defaultdict(int)

ids = xt_ids[xt_ids!=0]

mb = master_bar(range(TTA_N))
for tta in mb:
    test_preds = np.zeros((0, 29), dtype=np.float32)

    for batch_idx, batch in progress_bar(
        enumerate(learner.dl(DatasetType.Test)), total=len(learner.dl(DatasetType.Test)), parent=mb):
        _, preds_, _, _ = learner.pred_batch(ds_type=DatasetType.Test, batch=batch)
        preds_ = preds_.sum(dim=1)
        test_preds = np.concatenate([test_preds, preds_.data.cpu().numpy()], axis = 0)

    preds = test_preds[xt_ids!=0]
    for k in range(len(ids)):
        sub[int(ids[k])] += preds[k]
    
for k in range(len(ids)):
    sub[int(ids[k])] = sub[int(ids[k])]/TTA_N

sub_df = pd.DataFrame(sub.items(), columns=['id', 'scalar_coupling_constant'])
sub_df.head()

,id,scalar_coupling_constant
0,4658147,16.064374
1,4658148,193.402099
2,4658149,3.582070
3,4658150,186.372578
4,4658151,13.558205


In [30]:
sub_df.to_csv(sub_fname, index=False)

In [31]:
comp = 'champs-scalar-coupling'

In [32]:
!kaggle competitions submit -c {comp} -f {sub_fname} -m 'rot 1-hot aug feat_reg 1e-3 coulomb tta {TTA_N}'

100%|██████████████████████████████████████| 64.1M/64.1M [00:04<00:00, 14.6MB/s]
Successfully submitted to Predicting Molecular Properties

In [33]:
import time
time.sleep(60)
!kaggle competitions submissions -c {comp} -v > submissions-{comp}.csv


In [34]:
submissions = pd.read_csv(f'submissions-{comp}.csv')
submissions.iloc[0].publicScore

-1.94

In [ ]:
sub_fname